In [83]:
import numpy as np
import pdb
from gcp import bigquery as bq

In [89]:
def mape(X, predictions, y):
  num_timeslots = 43
  num_districts = 66
  if len(y.shape) == 1:
    y = np.asmatrix(y)
  if len(predictions.shape) == 1:
    predictions = np.asmatrix(predictions)
  Xy = np.concatenate((X, y.T, predictions.T), axis=1)
  districts = np.unique(X[:,0])
  district_scores = np.zeros(len(districts))
  for counter, key in enumerate(districts):
    group = np.compress((Xy[:,0] == key).flat, Xy, axis=0)
    district_scores[counter] = np.sum(
        (group[:,-2] -
         group[:,-1])/
        group[:,-2]
      ) / num_timeslots
  return np.sum(district_scores) / num_districts

def mape_scorer(estimator, X, y):
  estimator.fit(X)
  predictions = estimator.predict(X)
  return mape(X, predictions, y)

Testing MAPE

In [90]:
from sklearn.linear_model import LogisticRegression

est = LogisticRegression()
X = np.array([[1, 1], [1, 2], [2, 3], [2, 4]])
predictions = np.array([1, 2, 3, 4])
y = np.array([1, 2, 3, 4])

# Should return 0.0
mape(X, predictions, y)

0.0

In [84]:
%%sql --module q_all

SELECT *, HASH(CAST(district_id AS STRING) +timeslot) AS hash_value,
  IF(ABS(HASH(CAST(district_id AS STRING) + timeslot)) % 2 == 1, 'True', 'False')
    AS included_in_sample
FROM [datalab-projects-1331:xjk_algo_comp.gaps_processed]
WHERE gap > 0

In [86]:
query = bq.Query(q_all)
tableresult = query.results()

In [ ]:
# Reserve the first id as target ("gap").
features = ['gap', 'timeofday_slot', 'day_in_week', 'sum_price', 'traffic_tj_level1',
            'traffic_tj_level2', 'traffic_tj_level3', 'traffic_tj_level4']
all_data = np.zeros((tableresult.length, len(features)))
for rcounter, row in enumerate(tableresult):
  for fcounter, feature in enumerate(features):
    all_data[rcounter, fcounter] = row[feature]
  if rcounter % 5000 == 0:
    print 'processed {} rows'.format(rcounter)

processed 0 rows
processed 0 rows
processed 0 rows
processed 0 rows
processed 0 rows
processed 0 rows
processed 0 rows
processed 0 rows
processed 5000 rows
processed 5000 rows
processed 5000 rows
processed 5000 rows
processed 5000 rows
processed 5000 rows
processed 5000 rows
processed 5000 rows


In [88]:
tableresult.length

102680